<a href="https://colab.research.google.com/github/jagainu/SystemML_Usage/blob/master/lecture2_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2023年 大規模言語モデル サマースクール 第2回演習


## 演習の目的
「学習済みのLLM を（追加学習せずに）活用する技術について学ぶ」という目的のもと以下の目標を設定して講義を行いました。
- Prompting やIn-Context Learning とはなにか説明できるようになる。
- Augmented Language Model とはなにか説明できるようになる。

本演習では、Hugging Face を使った公開モデルを使った基本的なPromptingの実装、Augmented Language Models の1 種であるRetrieval Augmented Generation の実装を行います。

## 目次
- Prompting
    - 公開モデルをHuggingFace経由で使用
    - （補足) API経由でのみ利用可能なモデルを使用
    - Zero-shot / few-shot / CoT promptingでpromptingの効果を実感
    - Prompt engineering guide
- Retrieval Augmented Generation
    - Generation without Retrieval
    - Generation with gold passage
    - Retrieval の実装
    - Retrieval Augmented Generation


## Prompting
タスクに合わせてpromptを設計することで、追加の学習コストなしで大規模モデルを活用してタスクを解かせることができます。
重みにアクセスできないがAPI経由で使用できるClosed Model、重みにアクセスできるOpen Modelを用いてzero-shot / few-shot / CoT Promptingを試してみましょう。

### 公開モデルをHuggingFace経由で使用
API経由で使用できる大規模言語モデルは通常モデルの重みにはアクセスできず、自前データでの学習やモデルの分析などには限界があります。
研究機関や企業がモデルの重みを公開することもあり、1個人であっても大規模に訓練されたモデルの重みを活用して自由に使用することができます。(商用利用など特定の用途はライセンスによって制限される場合があります。)

先日Meta社から公開されたLlama2モデルは以下のリンクからリクエストを送信することで重みのダウンロードリンクを得ることができ、指定のライセンスのもと使用することができます。  
https://ai.meta.com/resources/models-and-libraries/llama-downloads/  
https://github.com/facebookresearch/llama  


HuggingFaceというプラットフォーム上で公開されることも多くあり、演習ではtransformersというライブラリを使用してHuggingFace上に公開されているモデルを使用してみます。  
https://huggingface.co/  
https://huggingface.co/docs/transformers/index  
HuggingFace上に公開されているモデルをtransformersライブラリを使用しロードし、ロードしたモデルにpromptを入れて出力を確認しましょう。

In [ ]:
import torch
# HuggingFaceにアップロードされたモデルやトークナイザーを使うためのライブラリ
from transformers import AutoTokenizer, AutoModelForCausalLM

# https://huggingface.co/rinna/bilingual-gpt-neox-4b
# https://huggingface.co/rinna/bilingual-gpt-neox-4b/tree/main
# from_pretrainedの引数にモデル名を指定すると、モデルをダウンロードしてきてくれます。ダウンロードには3分ほどかかります。
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

if torch.cuda.is_available():
    model = model.to("cuda")

In [ ]:
# ダウンロードしたファイルを確認、特に指定がない場合はhuggingfaceのモデルは~/.cache/huggingface/hubに保存されます。各インスタンス100GBがディスク容量の上限なので、モデルをダウンロードする前にディスク容量を確認してください。
!du -h /root/.cache/huggingface/hub/models--rinna--bilingual-gpt-neox-4b

In [ ]:
text = "大規模言語モデルについて説明してください。高校生でも理解できるように噛み砕いて説明してください。"
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=True,
        temperature=0.1,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

### (補足) API経由でのみ利用可能なモデルを使用
Google社のPALM2、OpenAI社のGPTシリーズ、Anthropic社のClaudeなど大規模言語モデルの重みにユーザーはアクセスできないが、API経由でpromptを送信し結果を受け取ることができます。自社で大規模言語モデルの推論サーバーをデプロイ・保守する必要なく手軽に使用することができるのが特徴です。

使用には料金が発生することがあり、使用するモデルや入力するpromptの長さ、出力される結果の長さによって料金が変わることがあります。

本演習ではOpenAIのAPIを使用してpromptingを試してみましょう。

各自OpenAIのアカウントを作成し、APIキーを取得してください。  
https://openai.com/blog/openai-api  
料金の目安  
https://openai.com/pricing#language-models  



In [ ]:
# copyした.envにAPI Keyを書き込む、terminalから.envへAPI keyを書き込んで保存してください。
!cp env.example .env

In [ ]:
import os
from dotenv import load_dotenv
# OpenAIのAPIを使うためのライブラリ
import openai
# OpenAIのtokenizerを使うためのライブラリ
import tiktoken
# API keyの取り扱いにはご注意ください。
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
# openai.api_key = "sk-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
# .envファイルにAPI keyを書いておくと、以下のように環境変数から読み込めます。
load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]
# https://platform.openai.com/docs/models/overview
openai_model_name = 'davinci-002'
openai_chat_model_name = 'gpt-3.5-turbo'

In [ ]:
prompt = "大規模言語モデルについて説明してください。高校生でも理解できるように噛み砕いて説明してください。"
# enc = tiktoken.encoding_for_model(openai_model_name)
enc = tiktoken.get_encoding("cl100k_base")
encoded_tokens = enc.encode(prompt)
print(len(encoded_tokens))
print(encoded_tokens)

In [ ]:
# 料金が発生します、実行の際はご注意ください。コメントアウトを外して実行すると、APIを使って文章を生成できます。
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0.1,
#   prompt=prompt
# )

# # Chat Completionの場合(gpt-3.5-turboなど)
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )


In [ ]:
# response

In [ ]:
# print(response.choices[0]['text'])

# print('入力prompt長: ', response.usage['prompt_tokens'])

# print('出力prompt長: ', response.usage['completion_tokens'])

In [ ]:
# print(chat_response.choices[0]['message']['content'])

# print('入力prompt長: ', chat_response.usage['prompt_tokens'])

# print('出力prompt長: ', chat_response.usage['completion_tokens'])

### Zero-shot / few-shot / CoT promptingでpromptingの効果を実感
promptingによって大規模言語モデルの出力が変化することを実感してみましょう。

JGLUEのタスクの一つであるJCommonSenseQAを例に、zero-shot / few-shot promptingの効果を確認してみます。

In [ ]:
# 日本語に特化した 60 億パラメータ規模の GPT モデルの構築と評価, https://www.anlp.jp/proceedings/annual_meeting/2023/pdf_dir/H9-4.pdf
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

In [ ]:
prompt = """
[問題]に対する[答え]を[選択肢]の中から選んでください。
[問題]:会社で一番偉い人はだれ?
[選択肢]:[社長, 部長, 人事部, 課長, エントリーシート]
[答え]:社長
[問題]:顔についていてものを食べるところは?
[選択肢]:[鼻, 目, 言葉, 口, 電話]
[答え]:口
[問題]:町より大きくて県より小さいものは何?
[選択肢]:[村, 役場, 市, 郡, 町内]
[答え]:市
[問題]:目標や手段や態度を一つに絞り、終始それで押し通そうとすること。また、そのさまを何という?
[選択肢]:[剣道, なぎなた, 牡丹槍, 一本槍, 管槍]
[答え]:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

few-shot promptingによって答えの単語のみ選ばれていることが確認でき、formattingの効果があることがわかります。  
次にCoT Promptingを試してみましょう。  
Llama2モデルを使用して、CoT promptingが提唱された論文中の例を試してみます。  
Llama2モデルの使用にはライセンスの同意の上、申請が必要なので各自申請をお願いします。  
https://ai.meta.com/resources/models-and-libraries/llama-downloads/  
上記の申請後以下からHuggingFaceでも申請を行う(Meta社への申請のメールアドレスとHuggingFaceのアカウントで登録しているメールアドレスが一致する必要あり)  
https://huggingface.co/meta-llama/Llama-2-13b-hf

In [ ]:
# 自分のHuggingFaceアカウントと紐付ける(申請済みのアカウントでないとモデルをダウンロードできないため)、terminalでhuggingface-cli loginを実行してください。
# !huggingface-cli login

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
# 前のセルまでに使用したGPUメモリを解放します。
del model
del tokenizer
torch.cuda.empty_cache()
# https://huggingface.co/meta-llama
# https://huggingface.co/docs/transformers/model_doc/llama2
# https://github.com/facebookresearch/llama-recipes/tree/main
# ダウンロードには6分ほどかかります。
model_name = 'meta-llama/Llama-2-13b-hf'
tokenizer = LlamaTokenizer.from_pretrained(model_name)

model =LlamaForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

In [ ]:
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))


In [ ]:
# Chain-of-Thought Prompting Elicits Reasoning in Large Language Models: https://arxiv.org/abs/2201.11903
prompt = """
Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.
Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.
Q: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
A: Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39. The answer is 39.
Q: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?
A: Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8. The answer is 8.
Q: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?
A: Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9. The answer is 9.
Q: There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?
A: There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29. The answer is 29.
Q: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?
A: Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls. The answer is 33.
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))

In [ ]:
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step. """

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=200)[0], skip_special_tokens=True))


「Let's think step by step」という文言によって思考の流れが出力されるようになりました。  
次にprompt中の些細な違いが出力にどのような影響を与えるかを確認してみましょう。

In [ ]:
print('############# with space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step."""

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=100)[0], skip_special_tokens=True))

print('############# with 全角space in the end #############')
prompt = """
Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
A: Let’s think step by step.　"""

model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, do_sample=False, max_new_tokens=200)[0], skip_special_tokens=True))

In [ ]:
# 料金が発生します
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step."""
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0,
#   prompt=prompt
# )
# print(response.choices[0]['text'])
# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])
# print('############# with space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step. """
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0,
#   prompt=prompt
# )
# print(response.choices[0]['text'])
# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])
# print('############# with 全角space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step.　"""
# response = openai.Completion.create(
#   model=openai_model_name,
#   max_tokens=100,
#   temperature=0,
#   prompt=prompt
# )
# print(response.choices[0]['text'])
# print('入力token長: ', response.usage['prompt_tokens'])
# print('出力token長: ', response.usage['completion_tokens'])
# print('total token長: ', response.usage['total_tokens'])


In [ ]:
# 料金が発生します
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step."""
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   temperature=0,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print('入力token長: ', chat_response.usage['prompt_tokens'])
# print('出力token長: ', chat_response.usage['completion_tokens'])
# print('total token長: ', chat_response.usage['total_tokens'])

# print('############# with space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step. """
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   temperature=0,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print('入力token長: ', chat_response.usage['prompt_tokens'])
# print('出力token長: ', chat_response.usage['completion_tokens'])
# print('total token長: ', chat_response.usage['total_tokens'])

# print('############# with 全角space in the end #############')
# prompt = """
# Q: Olivia has $23. She bought five bagels for $3 each. How much money does she have left?
# A: Let’s think step by step.　"""
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   temperature=0,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print('入力token長: ', chat_response.usage['prompt_tokens'])
# print('出力token長: ', chat_response.usage['completion_tokens'])
# print('total token長: ', chat_response.usage['total_tokens'])

### Prompt engineering guide
tokenizerや学習データの違いにより、使用するモデルによって、promptingの影響は異なります。  
選定したモデルによってどのような特徴があるのか、そのモデルが提唱された論文や公式ドキュメントを確認することで推し量ることができます。  
例えばOpenAI社のGPTシリーズでは以下のような事例集を公開しています。  
- https://github.com/openai/openai-cookbook
- https://platform.openai.com/docs/guides/gpt-best-practices
- https://help.openai.com/en/collections/3675942-prompt-engineering
    - https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

以下もおすすめです。  
- https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/
- https://github.com/dair-ai/Prompt-Engineering-Guide
- https://github.com/f/awesome-chatgpt-prompts

##  Retrieval Augmented Generation
言語モデルへpromptingを行うことで、幅広いタスクに対応するイメージを掴みました。  
さらなる活用方法として、言語モデルの論理思考能力をもとにしたサブタスク化を活用する方法や、言語モデル自身の重みだけでなく外部のツール・モデル・情報源を活用する方法があります。  
言語モデル単体だけでタスクを行わせる場合と比べて、より難しいタスクに対応できたり、より高い精度でタスクを解かせることができると期待されます。  
このような言語モデルの活用を[Augmented Language Model](https://arxiv.org/abs/2302.07842)と呼びます。  
Augmented Language Modelの一種であるRetrieval Augmented Generationについて実装し、言語モデルの活用方法についてイメージを掴みましょう。

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 前のセルまでに使用したGPUメモリを解放します。 これでも溢れてしまう場合はRestart Kernelを試してください。nvidia-smiコマンドでメモリ使用量を確認できます。
# del model
# del tokenizer
# torch.cuda.empty_cache()
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

if torch.cuda.is_available():
    model = model.to("cuda")

In [ ]:
!nvidia-smi

### Generation without Retrieval

In [ ]:
# In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083
prompt = """
以下の質問に回答してください:
質問:　東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？
回答:"""

token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

### Generation with gold passage

In [ ]:
# In-Context Retrieval-Augmented Language Models: https://arxiv.org/abs/2302.00083
query = "東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？"
retrieved_text = """
【東大松尾研 大規模言語モデル講座開講！】
 LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。
 """

prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に回答してください。
質問: {query}
回答:"""
print(prompt)

In [ ]:
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

In [ ]:
# OpenAIのAPIを利用した回答。料金がかかります。
# prompt = f"質問: {query}"
# print(prompt)
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])
# print("############# with retrieved_text #############")
# prompt = f"""{retrieved_text}
# 上記の文章に基づいて、質問に回答してください。
# 質問: {query}
# """
# print(prompt)
# chat_response = openai.ChatCompletion.create(
#   model=openai_chat_model_name,
#   messages=[
#         {"role": "user", "content": prompt},
#     ]
# )
# print(chat_response.choices[0]['message']['content'])

### Retrieval の実装
大規模モデルに質問をしただけでは事実に基づいた正しい回答が得られませんでしたが、回答の根拠となる文章が与えられると、根拠に基づいた回答がなされることを確認できました。

質問に応じて根拠となる文章を検索し、回答の際に参照することによって文を生成するのがRetrieval Augmented Generationです。  
[松尾研のニュース記事](https://weblab.t.u-tokyo.ac.jp/category/lab-news/)を回答根拠として質問に答える一連の流れを実装してみましょう。  
予め収集した記事データを読み込みます。(実装はcollect_news.pyを参照してください。)

In [ ]:
# 表形式のデータを読み込むためのライブラリ
import pandas as pd
df = pd.read_json('data.json')
df

どのように回答の根拠となる文章を選択するかにはさまざまな手法があります。  
今回は文章をベクトル化するモデルを使用し、ベクトルを元に質問文と記事中の文章の類似度を計算し、類似度が高い文章を選択することで回答の根拠となる文章を選択します。  

In [ ]:
# 文章や画像のembeddingを扱うためのライブラリ、HuggingFaceのモデルを使うこともできます。
from sentence_transformers import SentenceTransformer
# https://huggingface.co/intfloat/multilingual-e5-large
model = SentenceTransformer('intfloat/multilingual-e5-large')

In [ ]:
# 文章を1024次元のベクトルに変換する
embeddings = model.encode("東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？", normalize_embeddings=True)
print(embeddings.shape)
print(embeddings)

In [ ]:
input_texts = [
    "passage: 【東大松尾研 大規模言語モデル講座開講！】LLMを実装・活用するために必要な知識を扱う無償講座を9/4〜新規開講。GPTの基本的実装からInstruction Tuning/RLHF/高速化等最新のLLMを支える技術まで座学・演習を通じて体系的に学ぶ。募集対象は全国の学生。締切は7月末迄。",
    "passage: 【学生限定・受講生募集】9月27日から開講「世界モデルと知能」の受講生を募集します！本講座では，世界モデルを軸に最新の深層学習技術を身につけることができます．深層学習の基礎を習得済みであれば，東大以外の学生も応募可能です！ 締切：9月10日（日）23:59",
    "passage: 【学生限定・締切間近！】8/25(金)より開講！東大松尾研の金融系PJチームが企画・運営する短期集中講座「金融市場取引と機械学習」の受講生を募集。金融取引に対する機械学習の活用について、理論・実装の両面から学べます。締切:8/7(月)午前10時詳細:"
    ]
query_embeddings = model.encode(['query: 東京大学の松尾研究室が開講する大規模言語モデル講座ではどのような内容を扱いますか？'], normalize_embeddings=True)
passage_embeddings = model.encode(input_texts, normalize_embeddings=True)
print(passage_embeddings.shape)
# 類似度の計算
scores = (query_embeddings @ passage_embeddings.T) * 100
print(scores[0].tolist())
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
# 類似度の高い順のインデックスを取得
print(scores[0].argsort()[::-1])
# 一番高い類似度の文章を取得
print(input_texts[scores[0].argsort()[::-1][0]])

In [ ]:
input_texts = ['passage: ' + content for content in df.content.tolist()]
input_texts

### Retrieval Augmented Generation

In [ ]:
query = "松尾研ではどのようなロボティクス研究をしていますか？"
query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
passage_embeddings = model.encode(input_texts, normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100
# 上位3件を表示
print('score: ', scores[0][scores[0].argsort()[::-1][0]])
print(input_texts[scores[0].argsort()[::-1][0]])
print('score: ', scores[0][scores[0].argsort()[::-1][1]])
print(input_texts[scores[0].argsort()[::-1][1]])
print('score: ', scores[0][scores[0].argsort()[::-1][2]])
print(input_texts[scores[0].argsort()[::-1][2]])

In [ ]:
query = "松尾研ではどのようなロボティクス研究をしていますか？"
query_embeddings = model.encode(['query: ' + query], normalize_embeddings=True)
scores = (query_embeddings @ passage_embeddings.T) * 100

top_k = 2
top_k_idx = scores[0].argsort()[::-1][:top_k]


retrieved_text = f"""
{df.content.tolist()[top_k_idx[0]]}

{df.content.tolist()[top_k_idx[1]]}
"""

prompt = f"""{retrieved_text}
上記の文章に基づいて、質問に回答してください。
質問: {query}
回答:"""
print(prompt)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("rinna/bilingual-gpt-neox-4b", use_fast=False)
model = AutoModelForCausalLM.from_pretrained("rinna/bilingual-gpt-neox-4b")

if torch.cuda.is_available():
    model = model.to("cuda")
token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=300,
        min_new_tokens=100,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

どのような塊で回答根拠となるデータを蓄積するか、どのように回答根拠を選ぶか、どのように回答を生成するか、というそれぞれのステップで更なる工夫の余地があります。  
以下に参考資料を挙げておきます。
- https://github.com/openai/chatgpt-retrieval-plugin
    - https://github.com/openai/chatgpt-retrieval-plugin#limitations
    - https://github.com/openai/chatgpt-retrieval-plugin#future-directions
- https://techcommunity.microsoft.com/t5/azure-ai-services-blog/revolutionize-your-enterprise-data-with-chatgpt-next-gen-apps-w/ba-p/3762087
    - https://github.com/Azure-Samples/azure-search-openai-demo
- https://acl2023-retrieval-lm.github.io/